In [ ]:
%%bash
nrnivmodl

In [ ]:
# Download NEURON: http://www.neuron.yale.edu/neuron/download
# Download PyNeuronToolbox: https://github.com/ahwillia/PyNeuron-Toolbox

from neuron import h
import numpy as np
import math
import pylab as plt
import scipy.linalg
from PyNeuronToolbox.record import ez_record,ez_convert
#from PyNeuronToolbox import neuromorpho,morphology
from PyNeuronToolbox.morphology import shapeplot,shapeplot_animate,load
from mpl_toolkits.mplot3d import Axes3D
from JSAnimation import IPython_display
from matplotlib import animation
from matplotlib.pyplot import cm
import warnings
warnings.filterwarnings('ignore', category=SyntaxWarning)
%matplotlib inline
np.random.seed(123456789)

import re
import pandas as pd

DC = 10 # diffusion coeff [microns^2 s^-1]


In [ ]:
import sys
sys.path.append('../')
sys.path.append('../simulation/')

import sushibelt
import CA1_20reg_1dv_model as model
import time
N = model.N
tnorm = np.sum(model.target ** 2)
delta = 1e-5

In [ ]:
parval=pd.read_csv('../neuron-model/mprop_param.csv')
pval=parval[["F","Ctau","mProp","dvA","dvB","demand_CA1so_1","demand_CA1so_2","demand_CA1so_3","demand_CA1so_4","demand_CA1so_5","demand_CA1sr_1","demand_CA1sr_2","demand_CA1sr_3","demand_CA1sr_4","demand_CA1sr_5","demand_CA1sr_6","demand_CA1sr_7","demand_CA1sr_8","demand_CA1sr_9","demand_CA1sr_10","demand_CA1slm_1","demand_CA1slm_2","demand_CA1slm_3","demand_CA1slm_4","demand_CA1slm_5"]]
pval

In [ ]:
aL=list()
bL=list()
dL=list()
cL=list()
for i in range(pval.shape[0]):
    parr=np.array(pval.iloc[i])
    F = parr[0]
    Ctau = 10 ** parr[1]
    mProp = parr[2]
    dvA = parr[3]
    dvB = parr[4]
    dv = np.zeros(N)
    utarg = delta*np.ones(N)
    for k in range(N):
        if model.itarg[k] > 4:
            utarg[k] = parr[model.itarg[k]]
            dv[k] = (10 ** dvA) + (10 ** dvB)*utarg[k]
    utarg /= np.sum(utarg)
    K = np.sum(utarg) / N
    x = model.trafficking_solution(F * utarg + (1 - F) * K)
    x=np.append(x,0)
    a = (1 / (1 + x))
    aM, aF = sushibelt.aggregate_segments(np.concatenate((list(a),list(a))), model.segIdx, model.expD['Abbreviation'], fun=np.sum)
    aL.append(aM)
    b = list((1 / (1 + x ** -1)))
    bM, bF = sushibelt.aggregate_segments(np.concatenate((list(b),list(b))), model.segIdx, model.expD['Abbreviation'], fun=np.sum)
    bL.append(bM)
    dM, dF = sushibelt.aggregate_segments(np.concatenate((list(dv),list(dv))), model.segIdx, model.expD['Abbreviation'], fun=np.sum)
    dL.append(dM)
    c = Ctau * utarg / (F * utarg + (1 - F) * K)
    cM, cF = sushibelt.aggregate_segments(np.concatenate((list(c),list(c))), model.segIdx, model.expD['Abbreviation'], fun=np.sum)
    cL.append(cM)
(len(aL),len(bL),len(cL),len(dL),len(aL[0]))

In [ ]:
adf = pd.DataFrame(aL,columns=model.expD['Abbreviation'],index=[pval.iloc[k,2] for k in range(pval.shape[0])])
adf.to_csv('a_mprop.csv')
bdf = pd.DataFrame(bL,columns=model.expD['Abbreviation'],index=[pval.iloc[k,2] for k in range(pval.shape[0])])
bdf.to_csv('b_mprop.csv')
cdf = pd.DataFrame(cL,columns=model.expD['Abbreviation'],index=[pval.iloc[k,2] for k in range(pval.shape[0])])
cdf.to_csv('c_mprop.csv')
ddf = pd.DataFrame(dL,columns=model.expD['Abbreviation'],index=[pval.iloc[k,2] for k in range(pval.shape[0])])
ddf.to_csv('d_mprop.csv')

In [ ]:
aL=list()
bL=list()
dL=list()
cL=list()
for i in range(pval.shape[0]):
    parr=np.array(pval.iloc[i])
    F = parr[0]
    Ctau = 10 ** parr[1]
    mProp = parr[2]
    dvA = parr[3]
    dvB = parr[4]
    dv = np.zeros(N)
    utarg = delta*np.ones(N)
    for k in range(N):
        if model.itarg[k] > 4:
            utarg[k] = parr[model.itarg[k]]
            dv[k] = (10 ** dvA) + (10 ** dvB)*utarg[k]
    utarg /= np.sum(utarg)
    K = np.sum(utarg) / N
    x = model.trafficking_solution(F * utarg + (1 - F) * K)
    x=np.append(x,0)
    a = (1 / (1 + x))
    aM, aF = sushibelt.aggregate_segments(np.concatenate((list(a),list(a))), model.segIdx, model.expD['Abbreviation'], fun=np.mean)
    aL.append(aM)
    b = list((1 / (1 + x ** -1)))
    bM, bF = sushibelt.aggregate_segments(np.concatenate((list(b),list(b))), model.segIdx, model.expD['Abbreviation'], fun=np.mean)
    bL.append(bM)
    dM, dF = sushibelt.aggregate_segments(np.concatenate((list(dv),list(dv))), model.segIdx, model.expD['Abbreviation'], fun=np.mean)
    dL.append(dM)
    c = Ctau * utarg / (F * utarg + (1 - F) * K)
    cM, cF = sushibelt.aggregate_segments(np.concatenate((list(c),list(c))), model.segIdx, model.expD['Abbreviation'], fun=np.mean)
    cL.append(cM)
(len(aL),len(bL),len(cL),len(dL),len(aL[0]))

In [ ]:
adf = pd.DataFrame(aL,columns=model.expD['Abbreviation'],index=[pval.iloc[k,2] for k in range(pval.shape[0])])
adf.to_csv('a_mprop.mean.csv')
bdf = pd.DataFrame(bL,columns=model.expD['Abbreviation'],index=[pval.iloc[k,2] for k in range(pval.shape[0])])
bdf.to_csv('b_mprop.mean.csv')
cdf = pd.DataFrame(cL,columns=model.expD['Abbreviation'],index=[pval.iloc[k,2] for k in range(pval.shape[0])])
cdf.to_csv('c_mprop.mean.csv')
ddf = pd.DataFrame(dL,columns=model.expD['Abbreviation'],index=[pval.iloc[k,2] for k in range(pval.shape[0])])
ddf.to_csv('d_mprop.mean.csv')

In [ ]:
len(model.parnames)

In [ ]:
aM, aF = sushibelt.aggregate_segments(np.concatenate((aL[0],aL[0])), model.segIdx, model.expD['Abbreviation'], fun=np.sum)

In [ ]:
x=np.array(range(model.expD.shape[0]),dtype=int)
fig, ax = plt.subplots()
ax.plot(x,aM,label='a')
#ax.plot(x,resF/(1-mProp),label='norm detached DC')
ax.set_xlabel('Region number')
ax.set_ylabel('Value')
ax.set_xticks(x,model.expD['Abbreviation'],rotation=90)
# show a legend on the plot
ax.legend()
#plt.show()
pos = ax.get_position()
ax.set_position([pos.x0, pos.y0 + pos.height*0.15, pos.width , pos.height* 0.85])
plt.title("A[0]") 
# function to show the plot
plt.show()
